In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('../../embedded_output.parquet')

In [ ]:
df.shape

In [ ]:
df = df.dropna(subset=['skill_code']).copy()

df = df[df['correctness'] <= 1].copy()
df = df[df['correctness'] >= 0].copy()

mask = df.answer_text.str.contains(r"[A-Za-z]", na=False)
df = df[mask].copy() # must include a letter

print('after removing non letter')
print(df.shape)

mask.value_counts()

mask_words = df.answer_text.str.split().str.len() >= 10
df = df[mask_words].copy()

print('after removing non 10 words')
print(df.shape)

mask_words.value_counts()

s = df.groupby('teacher_id')['correctness'].std()
df = df[df['teacher_id'].isin(s[s.notna() & (s != 0)].index)]

print('after removing teachers with no grading variance')
print(df.shape)

df.shape

import re
mask_img = ~df.answer_text.str.fullmatch(r"\s*(<p>\s*)?<img[^>]*>\s*(</p>\s*)?", flags=re.I)
df = df[mask_img].copy()

print('after removing images')
print(df.shape)

In [ ]:
len(set(df.teacher_id))

In [ ]:
len(set(df.student_user_id))

In [ ]:
import numpy as np, pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score
from tqdm import tqdm

df_sorted=df.sort_values("problem_start_time").reset_index(drop=True)
g_t=df_sorted.groupby("teacher_id")["correctness"]
df_sorted["teacher_prior"]=((g_t.cumsum()-df_sorted["correctness"])/g_t.cumcount().replace(0,np.nan)).fillna(0)

cut=int(len(df_sorted)*0.8)
train,test=df_sorted.iloc[:cut],df_sorted.iloc[cut:]

resp_tr,prob_tr=np.vstack(train.embedding_response),np.vstack(train.embedding_problembody)
resp_te,prob_te=np.vstack(test.embedding_response),np.vstack(test.embedding_problembody)
ytr,yte=train.correctness.values,test.correctness.values
median_val=df_sorted.correctness.median()

tp_tr,tp_te=train.teacher_prior.values.reshape(-1,1),test.teacher_prior.values.reshape(-1,1)

centroids=pd.DataFrame(resp_tr).groupby(train.problem_body).mean()
cent_tr=centroids.reindex(train.problem_body).values
cent_te=centroids.reindex(test.problem_body).fillna(0).values

def run(name,Xtr,Xte):
    m=LassoCV(alphas=np.logspace(-3,0,10),cv=3,max_iter=5000,n_jobs=-1,precompute=False).fit(Xtr,ytr)
    p=np.clip(m.predict(Xte),0,1)
    return dict(model=name,alpha=m.alpha_,p=p)

variants=[
    ("teacher_prior_only",tp_tr,tp_te),
    ("resp_only",resp_tr,resp_te),
    ("problem_only",prob_tr,prob_te),
    ("problem+response",np.hstack([prob_tr,resp_tr]),np.hstack([prob_te,resp_te])),
    ("resp_minus_prob",resp_tr-prob_tr,resp_te-prob_te),
    ("resp_minus_respCentroidByProblem",resp_tr-cent_tr,resp_te-cent_te),
    ("teacher_prior+resp",np.hstack([tp_tr,resp_tr]),np.hstack([tp_te,resp_te])),
    ("teacher_prior+resp_minus_prob",np.hstack([tp_tr,resp_tr-prob_tr]),np.hstack([tp_te,resp_te-prob_te])),
    ("teacher_prior+resp_minus_respCentroidByProblem",np.hstack([tp_tr,resp_tr-cent_tr]),np.hstack([tp_te,resp_te-cent_te])),
]

res=[run(n,Xtr,Xte) for n,Xtr,Xte in tqdm(variants)]
bin_y=(yte>=median_val).astype(int)
idx=np.random.randint(0,len(yte),(1000,len(yte)))

In [ ]:
rows=[]
for r in tqdm(res):
    p=r["p"]
    r2=r2_score(yte,p); mse=mean_squared_error(yte,p)
    try: auc=roc_auc_score(bin_y,p)
    except: auc=np.nan
    r2_b=np.array([r2_score(yte[i],p[i]) for i in idx])
    mse_b=np.array([mean_squared_error(yte[i],p[i]) for i in idx])
    auc_b=[]
    for i in idx:
        try: auc_b.append(roc_auc_score(bin_y[i],p[i]))
        except: pass
    auc_b=np.array(auc_b) if len(auc_b)>0 else np.array([np.nan])
    rows.append(dict(model=r["model"],alpha=r["alpha"],
                     r2=r2,r2_lo=np.percentile(r2_b,2.5),r2_hi=np.percentile(r2_b,97.5),
                     mse=mse,mse_lo=np.percentile(mse_b,2.5),mse_hi=np.percentile(mse_b,97.5),
                     auc=auc,auc_lo=np.nanpercentile(auc_b,2.5),auc_hi=np.nanpercentile(auc_b,97.5)))
df_ci=pd.DataFrame(rows).sort_values("model").reset_index(drop=True)

In [ ]:
print(pd.DataFrame(df_ci).sort_values("auc",ascending=False).drop(columns=['r2', 'alpha', 'r2_lo', 'r2_hi']))

## Export for manual inspection

In [ ]:
import numpy as np, pandas as pd

names=["teacher_prior+resp","resp_only","teacher_prior_only"]
preds={r["model"]:r["p"] for r in res if r["model"] in names}

Z=pd.DataFrame({
    "row_idx":np.arange(len(yte)),
    "teacher_id":test["teacher_id"].to_numpy(),
    "problem_id":test["problem_body"].to_numpy(),
    "answer_text":test["answer_text"].to_numpy(),
    "correctness":yte,
    "timestamp":test["problem_start_time"].to_numpy(),
    "pred_tp_plus_resp":preds["teacher_prior+resp"],
    "pred_resp_only":preds["resp_only"],
    "pred_tp_only":preds["teacher_prior_only"],
})

P=np.vstack([
    Z["pred_tp_plus_resp"].values,
    Z["pred_resp_only"].values,
    Z["pred_tp_only"].values
]).T

cands=np.linspace(0,1,101)
disagree=lambda B: (B.max(axis=1)!=B.min(axis=1)).sum()
thr=cands[int(np.argmax([disagree((P>=t).astype(int)) for t in cands]))]

Z["bin_tp_plus_resp"]=(Z["pred_tp_plus_resp"]>=thr).astype(int)
Z["bin_resp_only"]=(Z["pred_resp_only"]>=thr).astype(int)
Z["bin_tp_only"]=(Z["pred_tp_only"]>=thr).astype(int)

Z=Z[Z[["bin_tp_plus_resp","bin_resp_only","bin_tp_only"]].nunique(axis=1)>1].reset_index(drop=True)

Z["binary_preds_pattern"]=Z[["bin_tp_plus_resp","bin_resp_only","bin_tp_only"]].astype(str).agg("-".join,axis=1)
Z["binary_preds_pattern_readable"]=(
    "TP+Resp="+Z["bin_tp_plus_resp"].astype(str)+
    " | Resp-only="+Z["bin_resp_only"].astype(str)+
    " | TP-only="+Z["bin_tp_only"].astype(str)
)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

E_df = pd.DataFrame(resp_te[Z["row_idx"].to_numpy()])
E_df["binary_preds_pattern"] = Z["binary_preds_pattern"].to_numpy()
centroids = E_df.groupby("binary_preds_pattern").mean(numeric_only=True)
Z["prototype_score"] = cosine_similarity(
    E_df.drop(columns="binary_preds_pattern").values,
    centroids.loc[Z["binary_preds_pattern"]].values
)[:,0]

Z = Z.drop_duplicates(subset=["answer_text"]).reset_index(drop=True)

m = Z["binary_preds_pattern"].nunique()
k_high = 150 // m
k_low = 150 // m

grp = Z.groupby("binary_preds_pattern", group_keys=False)
S_high = grp.apply(lambda g: g.nlargest(k_high, "prototype_score"))
S_low = grp.apply(lambda g: g.nsmallest(k_low, "prototype_score"))

rem_high = 150 - len(S_high)
rem_low = 150 - len(S_low)
pool = Z.drop(S_high.index.union(S_low.index), errors="ignore")
S_high = pd.concat([S_high, pool.nlargest(rem_high, "prototype_score")]) if rem_high > 0 else S_high
pool = pool.drop(S_high.index, errors="ignore")
S_low = pd.concat([S_low, pool.nsmallest(rem_low, "prototype_score")]) if rem_low > 0 else S_low

S = pd.concat([S_high, S_low]).drop_duplicates(subset=["answer_text"]).head(300)

S = S[[
    "row_idx","teacher_id","problem_id","timestamp","answer_text","correctness",
    "pred_tp_plus_resp","pred_resp_only","pred_tp_only",
    "bin_tp_plus_resp","bin_resp_only","bin_tp_only",
    "binary_preds_pattern","binary_preds_pattern_readable","prototype_score"
]].copy()

for col in ["theme_reason_for_difference","theme_strategy_pattern","theme_student_factor","theme_teacher_factor","coder_notes"]:
    S[col] = ""

S.to_csv("lak26-representations-coding_sample.csv", index=False)
S

In [ ]:
Z.binary_preds_pattern.value_counts()

In [ ]:
S.binary_preds_pattern.value_counts()

## Interpretability

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.linear_model import LassoCV

m_resp=LassoCV(alphas=np.logspace(-3,0,10),cv=3,max_iter=5000,n_jobs=-1,precompute=False).fit(resp_tr,ytr)
m_tonly=LassoCV(alphas=np.logspace(-3,0,10),cv=3,max_iter=5000,n_jobs=-1,precompute=False).fit(tp_tr,ytr)
Xtr=np.hstack([tp_tr,resp_tr]); Xte=np.hstack([tp_te,resp_te])
m_adj=LassoCV(alphas=np.logspace(-3,0,10),cv=3,max_iter=5000,n_jobs=-1,precompute=False).fit(Xtr,ytr)

def resp_coefs(model):
    c=model.coef_
    return c if c.size==resp_tr.shape[1] else c[-resp_tr.shape[1]:]

def summarize(mode, coefs):
    nz=np.flatnonzero(coefs); k=len(nz); pct=100*k/len(coefs)
    print(f"[{mode}] non-zero: {k}/{len(coefs)} ({pct:.1f}%)")
    if k:
        top=np.argsort(np.abs(coefs[nz]))[::-1][:100]
        idx=nz[top]; w=coefs[idx]
        print(f"[{mode}] top dims (idx,wt):"); [print(f"{int(i):4d}\t{wj:+.6f}") for i,wj in zip(idx,w)]
    return nz

def proj_and_resid_unadjusted():
    c=resp_coefs(m_resp); nz=summarize("unadjusted",c)
    proj=resp_te[:,nz]@c[nz]
    resid=yte-m_resp.predict(resp_te)
    return proj,resid,c,nz

def proj_and_resid_adjusted():
    c=resp_coefs(m_adj); nz=summarize("adjusted",c)
    proj=resp_te[:,nz]@c[nz]
    resid=yte-m_tonly.predict(tp_te)
    return proj,resid,c,nz

proj_u,resid_u,c_u,nz_u=proj_and_resid_unadjusted()
proj_a,resid_a,c_a,nz_a=proj_and_resid_adjusted()

n=len(yte)
idx=np.random.choice(n,n,replace=False)
sns.kdeplot(x=proj_u[idx],y=resid_u[idx],fill=True,cmap="viridis"); plt.xlabel("Projection (resp-only coefs)"); plt.ylabel("Residual (resp-only)"); plt.title("Embedding signals (unadjusted)"); plt.savefig("embedding_signals_unadjusted.pdf",format="pdf",bbox_inches="tight"); plt.show()
sns.kdeplot(x=proj_a[idx],y=resid_a[idx],fill=True,cmap="viridis"); plt.xlabel("Projection (teacher+resp coefs)"); plt.ylabel("Residual (teacher-only)"); plt.title("Embedding signals (adjusted)"); plt.savefig("embedding_signals_adjusted.pdf",format="pdf",bbox_inches="tight"); plt.show()

def show_dim(mode="adjusted",rank=0,k=3):
    if mode=="adjusted":
        coefs=c_a; nz=nz_a; dim=nz[np.argsort(np.abs(coefs[nz]))[::-1][rank]]
        proj=resp_te[:,dim]; pred=m_adj.predict(np.hstack([tp_te,resp_te])).clip(0,1); resid=yte-pred
    else:
        coefs=c_u; nz=nz_u; dim=nz[np.argsort(np.abs(coefs[nz]))[::-1][rank]]
        proj=resp_te[:,dim]; pred=m_resp.predict(resp_te).clip(0,1); resid=yte-pred
    df_ex=pd.DataFrame({"dimension":dim,"weight":coefs[dim],"projection":proj,"prediction":pred,"residual":resid,"grade":yte,"teacher_id":test.teacher_id.values,"problem_body":test.problem_body.values,"answer_text":test.answer_text.values}).drop_duplicates(subset=["answer_text"])
    lo=df_ex.nsmallest(k,"projection"); hi=df_ex.nlargest(k,"projection")
    print(f"=== [{mode}] Dimension {dim} weight={coefs[dim]:+.6f} ===\n-- Low --")
    for _,r in lo.iterrows():
        print(f"Teacher:{r.teacher_id} Grade:{r.grade:.3f} Pred:{r.prediction:.3f} Resid:{r.residual:.3f} Proj:{r.projection:.3f}\nProblem: {r.problem_body}\nAnswer : {r.answer_text}\n")
    print("-- High --")
    for _,r in hi.iterrows():
        print(f"Teacher:{r.teacher_id} Grade:{r.grade:.3f} Pred:{r.prediction:.3f} Resid:{r.residual:.3f} Proj:{r.projection:.3f}\nProblem: {r.problem_body}\nAnswer : {r.answer_text}\n")

for i in range(min(20,len(nz_a))): show_dim("adjusted",i,3)
for i in range(min(20,len(nz_u))): show_dim("unadjusted",i,3)